<a href="https://colab.research.google.com/github/RodrigoZonzin/grafos/blob/main/colonia_formigas/TP_grafos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install igraph

In [ ]:
import random, math
import numpy as np
import matplotlib.pyplot as plt
import igraph as ig


In [ ]:
params = {
		"num_formigas": 8,
		"alfa": 1.0,
		"beta": 5.0,
		"iteracoes": 1000,
		"evaporacao": 0.5,
}

In [ ]:
def inicializa_ACO(grafo, num_formigas, alfa=1.0, beta=5.0, iteracoes=10, evaporacao=0.5):
    formigas = []
    lista_cidades = [cidade for cidade in range(1, grafo.num_vertices + 1)]

    for k in range(num_formigas):
        cidade_formiga = random.choice(lista_cidades)
        lista_cidades.remove(cidade_formiga)
        formigas.append(Formiga(cidade = cidade_formiga))
        if not lista_cidades:
            lista_cidades = [cidade for cidade in range(1, grafo.num_vertices + 1)]

    custo_guloso = 0.0
    vertice_inicial = random.randint(1, grafo.num_vertices)
    vertice_corrente = vertice_inicial
    visitados = [vertice_corrente]

    while True:
        vizinhos = grafo.vizinhos[vertice_corrente][:]
        custos, escolhidos = [], {}

        for vizinho in vizinhos:
            if vizinho not in visitados:
                custo = grafo.obterCustoAresta(vertice_corrente, vizinho)
                escolhidos[custo] = vizinho
                custos.append(custo)

        if len(visitados) == grafo.num_vertices:
            break

        min_custo = min(custos)
        custo_guloso += min_custo
        vertice_corrente = escolhidos[min_custo]
        visitados.append(vertice_corrente)

    custo_guloso += grafo.obterCustoAresta(visitados[-1], vertice_inicial)

    for chave_aresta in grafo.arestas:
        feromonio = 1.0 / (grafo.num_vertices * custo_guloso)
        grafo.setFeromonioAresta(chave_aresta[0], chave_aresta[1], feromonio)

    return formigas

In [ ]:
def rodar_ACO(grafo, formigas, num_formigas, alfa=1.0, beta=5.0, iteracoes=10, evaporacao=0.5):
    for it in range(iteracoes):
        cidades_visitadas = []

        for k in range(num_formigas):
            cidades = [formigas[k].obterCidade()]
            cidades_visitadas.append(cidades)

        for k in range(num_formigas):
            for i in range(1, grafo.num_vertices):
                cidades_nao_visitadas = list(set(grafo.vizinhos[formigas[k].obterCidade()]) - set(cidades_visitadas[k]))
                somatorio = 0.0

                for cidade in cidades_nao_visitadas:
                    feromonio = grafo.obterFeromonioAresta(formigas[k].obterCidade(), cidade)
                    distancia = grafo.obterCustoAresta(formigas[k].obterCidade(), cidade)

                    somatorio += (math.pow(feromonio, alfa) * math.pow(1.0 / distancia, beta))

                probabilidades = {}

                for cidade in cidades_nao_visitadas:
                    feromonio = grafo.obterFeromonioAresta(formigas[k].obterCidade(), cidade)
                    distancia = grafo.obterCustoAresta(formigas[k].obterCidade(), cidade)
                    probabilidade = (math.pow(feromonio, alfa) * math.pow(1.0 / distancia, beta)) / (somatorio if somatorio > 0 else 1)
                    probabilidades[cidade] = probabilidade

                cidade_escolhida = max(probabilidades, key=probabilidades.get)
                cidades_visitadas[k].append(cidade_escolhida)

            formigas[k].setSolucao(cidades_visitadas[k], grafo.obterCustoCaminho(cidades_visitadas[k]))


        for aresta in grafo.arestas:
            somatorio_feromonio = 0.0

            for k in range(num_formigas):
                arestas_formiga = []

                for j in range(grafo.num_vertices - 1):
                    arestas_formiga.append((cidades_visitadas[k][j], cidades_visitadas[k][j+1]))

                arestas_formiga.append((cidades_visitadas[k][-1], cidades_visitadas[k][0]))

                if aresta in arestas_formiga:
                    somatorio_feromonio += (1.0 / grafo.obterCustoCaminho(cidades_visitadas[k]))

            novo_feromonio = (1.0 - evaporacao) * grafo.obterFeromonioAresta(aresta[0], aresta[1]) + somatorio_feromonio
            grafo.setFeromonioAresta(aresta[0], aresta[1], novo_feromonio)

    solucao, custo = None, None

    for k in range(num_formigas):
        if not solucao:
            solucao = formigas[k].obterSolucao()[:]
            custo = formigas[k].obterCustoSolucao()
        else:
            aux_custo = formigas[k].obterCustoSolucao()
            if aux_custo < custo:
                solucao = formigas[k].obterSolucao()[:]
                custo = aux_custo

    #solucao = [i+1 for i in solucao]
    return (solucao, custo)

In [ ]:
def int_to_alpha(n):
    return chr(ord('A') + n - 1)

In [ ]:
if __name__ == "__main__":

  with open('arquivo.txt', 'r') as file:
    matriz_distancias = [[int(num) for num in line.split()] for line in file]

  num_vertices = len(matriz_distancias)
  grafo = Grafo(num_vertices)

  for i in range(num_vertices):
    for j in range(num_vertices):
        if i != j:
            grafo.adicionarAresta(i, j, matriz_distancias[i][j])



  G = ig.Graph()
  G.add_vertices(range(1, 16))
  arestas, pesos = grafo.getAresta()
  #arestas = [tuple(int_to_alpha(num) for num in tup) for tup in arestas]
  G.add_edges(arestas)
  G.es["weight"] = pesos

  #fig, ax = plt.subplots()
  #ig.plot(G, target=ax, vertex_label=G.vs["name"]) #, edge_label = G.es["weight"])
  #plt.plot()

  custos_analise = []
  for i in range(10, 2000, 10):
    params['num_vertices'] = num_vertices
    formigas = inicializa_ACO(grafo, 7, alfa = 1, beta = 5, iteracoes = i, evaporacao = 0.5)
    sol, custo = rodar_ACO(grafo, formigas, 7, alfa=1, beta=5, iteracoes=i, evaporacao=0.5)
    custos_analise.append(custo)



  #G_cv = ig.Graph()
  #G_cv.add_vertices(range(1, 16))
  #G_cv.add_edges(arestas)


  #for i in range(len(sol)-1):
  #  edge_index = G_cv.get_eid(sol[i], sol[i+1], directed=False)
  #  G_cv.es[edge_index]["color"] = "yellow"


  #fig, ax = plt.subplots()
  #ig.plot(G_cv, target=ax, vertex_label=G_cv.vs["name"])
  #plt.title(f"Caminho{[str(i) for i in sol]}\nCusto:{custo}")
  #plt.plot()


KeyError: ignored

In [ ]:
plt.plot(custos_analise, color = 'green')
#plt.grid(True)
plt.xlabel("Iteração")
plt.ylabel("Custo")
#plt.title("")
#plt.ylim(480)

In [ ]:
custos_analise = np.array(custos_analise)

print(np.max(custos_analise),
  custos_analise.shape,
  np.min(custos_analise),
  np.mean(custos_analise),
  np.std(custos_analise),
  np.quantile(custos_analise, 0.10))

In [ ]:
class Aresta:

	def __init__(self, origem, destino, custo):
		self.origem = origem
		self.destino = destino
		self.custo = custo
		self.feromonio = None

	def obterOrigem(self):
		return self.origem

	def obterDestino(self):
		return self.destino

	def obterCusto(self):
		return self.custo

	def obterFeronomio(self):
		return self.feromonio

	def setFeromonio(self, feromonio):
		self.feromonio = feromonio


# classe que representa um grafo (grafos completos)
class Grafo:

	def __init__(self, num_vertices):
		self.num_vertices = num_vertices # número de vértices do grafo
		self.arestas = {} # dicionário com as arestas
		self.vizinhos = {} # dicionário com todos os vizinhos de cada vértice


	def adicionarAresta(self, origem, destino, custo):
		aresta = Aresta(origem=origem, destino=destino, custo=custo)
		self.arestas[(origem, destino)] = aresta
		if origem not in self.vizinhos:
			self.vizinhos[origem] = [destino]
		else:
			self.vizinhos[origem].append(destino)

	def getAresta(self):
		arestas = []
		pesos = []
		for aresta in self.arestas:
			arestas.append(aresta)
			pesos.append(self.obterCustoAresta(aresta[0], aresta[1]))
		return arestas, pesos


	def obterCustoAresta(self, origem, destino):
		return self.arestas[(origem, destino)].obterCusto()

	def obterFeromonioAresta(self, origem, destino):
		return self.arestas[(origem, destino)].obterFeronomio()

	def setFeromonioAresta(self, origem, destino, feromonio):
		self.arestas[(origem, destino)].setFeromonio(feromonio)

	def obterCustoCaminho(self, caminho):
		custo = 0
		for i in range(self.num_vertices - 1):
			custo += self.obterCustoAresta(caminho[i], caminho[i+1])
		# adiciona o último custo
		custo += self.obterCustoAresta(caminho[-1], caminho[0])
		return custo

# classe que representa uma formiga
class Formiga:

	def __init__(self, cidade):
		self.cidade = cidade
		self.solucao = []
		self.custo = None

	def obterCidade(self):
		return self.cidade

	def setCidade(self, cidade):
		self.cidade = cidade

	def obterSolucao(self):
		return self.solucao

	def setSolucao(self, solucao, custo):
		# atualiza a solução
		if not self.custo:
			self.solucao = solucao[:]
			self.custo = custo
		else:
			if custo < self.custo:
				self.solucao = solucao[:]
				self.custo = custo

	def obterCustoSolucao(self):
		return self.custo
